In [3]:
# Desafío - Transformación y manipulación de datos (Parte II)

In [4]:
# 1. Carga los datos de cada pestaña del archivo US_Regional_Sales_Data.xlsx en un DataFrame independiente.
# (hint: Puedes obtener los nombres de las hojas usando pd.ExcelFile(archivo).sheet_names)

import pandas as pd
import numpy as np


archivo = 'US_Regional_Sales_Data.xlsx'   # Obtengo el archivo.
excel_file = pd.ExcelFile(archivo)        # Creo un objeto ExcelFile.
sheets = excel_file.sheet_names           # Extraigo los nombres de las pestañas en la variable sheets.


dataframes = {}                           # Creo un diccionario para almacenar cada hoja como un DataFrame independiente.
for sheet in sheets:
    dataframes[sheet] = excel_file.parse(sheet)     # Cargo cada hoja en un DataFrame y lo guardo en el diccionario.

for sheet in sheets:
    var_name = f"df_{sheet.replace(' ', '_')}"   # Creo el nombre variable sin espacios.
    globals()[var_name] = dataframes[sheet]      # Creo variable global con ese nombre y asigno el DataFrame.

print(dataframes.keys())                         # Verifico los nombres de los nuevos df.

dict_keys(['Sales Orders Sheet', 'Customers Sheet', 'Store Locations Sheet', 'Products Sheet', 'Sales Team Sheet'])


In [5]:
# 2. Cruza todos los DataFrames usando validación many_to_one, y guarda el resultado en una nueva variable llamada df_base.

df_base = df_Sales_Orders_Sheet.merge(          # Parto con la tabla principal de órdenes de venta.
    df_Customers_Sheet,
    how='left',
    on='_CustomerID',
    validate='many_to_one'
)

df_base = df_base.merge(                        # Uno con la tabla de ubicaciones de tiendas para agregar info de la tienda donde se realizó la venta.
    df_Store_Locations_Sheet,
    how='left',
    on='_StoreID',
    validate='many_to_one'
)

df_base = df_base.merge(                        # Uno con la tabla de productos para agregar info de los productos vendidos.
    df_Products_Sheet,
    how='left',
    on='_ProductID',
    validate='many_to_one'
)

df_base = df_base.merge(                        # Uno con la tabla de categorías de productos para agregar info de las categorías de los productos vendidos.
    df_Sales_Team_Sheet,
    how='left',
    on='_SalesTeamID',
    validate='many_to_one'
)

df_base.head()                                  # Muestro las primeras filas del DataFrame resultante para verificar la union.

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,...,AreaCode,Population,Household Income,Median Income,Land Area,Water Area,Time Zone,Product Name,Sales Team,Region
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,...,631,213776,68789,80327,135481314,160302131,America/New York,Dining Furniture,Joshua Bennett,Northeast
1,SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,...,913,186515,74830,72463,194562941,1309517,America/Chicago,Wreaths,Paul Holmes,Midwest
2,SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,...,734,117070,47179,55990,72727009,2254528,America/Detroit,Stemware,Samuel Fowler,Midwest
3,SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,...,203,130322,49771,37192,48407373,3739487,America/New York,Accessories,Carlos Miller,West
4,SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,...,203,108802,40213,40467,73880017,1086045,America/New York,Candles,Joe Price,Northeast


In [6]:
# 3. Agrega las siguientes columnas a df_ base.

#   Columna                              Cálculo

# Procurement                  Days OrderDate - ProcuredDate
# Shipping                     Days ShipDate - OrderDate
# DeliveryDays                 DeliveryDate - ShipDate
# CustomerDays                 ShippingDays + DeliveryDays

# hint: Asegúrate de que las columnas sean de tipo datetime.


# Convierto primero todas las columnas de fecha a tipo datetime.

df_base['OrderDate'] = pd.to_datetime(df_base['OrderDate'])
df_base['ProcuredDate'] = pd.to_datetime(df_base['ProcuredDate'])
df_base['ShipDate'] = pd.to_datetime(df_base['ShipDate'])
df_base['DeliveryDate'] = pd.to_datetime(df_base['DeliveryDate'])

# # Calculo de tiempos (en dias) entre etapas clave del proceso de compra y entrega.

df_base['Procurement'] = (df_base['OrderDate'] - df_base['ProcuredDate']).dt.days
df_base['Shipping'] = (df_base['ShipDate'] - df_base['OrderDate']).dt.days
df_base['DeliveryDays'] = (df_base['DeliveryDate'] - df_base['ShipDate']).dt.days
df_base['CustomerDays'] = df_base['Shipping'] + df_base['DeliveryDays']

df_base.head()   # Muestro las primeras filas del DataFrame resultante para verificar los calculos.

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,...,Land Area,Water Area,Time Zone,Product Name,Sales Team,Region,Procurement,Shipping,DeliveryDays,CustomerDays
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,...,135481314,160302131,America/New York,Dining Furniture,Joshua Bennett,Northeast,151,14,5,19
1,SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,...,194562941,1309517,America/Chicago,Wreaths,Paul Holmes,Midwest,151,22,10,32
2,SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,...,72727009,2254528,America/Detroit,Stemware,Samuel Fowler,Midwest,151,21,10,31
3,SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,...,48407373,3739487,America/New York,Accessories,Carlos Miller,West,151,2,5,7
4,SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,...,73880017,1086045,America/New York,Candles,Joe Price,Northeast,51,16,10,26


In [7]:
# 4. Agrega una nueva columna discreta al df_base, con el nombre CustomerDaysInterval que clasifique los valores de la columna CustomerDays en los siguientes intervalos:

# 0 to 15 days
# 15 to 30 days
# 30 to 45 days
# 45 to 60 days
# 60 to 75 days
# 75 to 90 days

# Creo la columna 'CustomerDaysInterval' clasificando 'CustomerDays' en rangos de 15 días con etiquetas personalizadas.
bins = [0, 15, 30, 45, 60, 75, 90]
labels = ['0-15', '15-30', '30-45', '45-60', '60-75', '75-90']

df_base['CustomerDaysInterval'] = pd.cut(
    df_base['CustomerDays'],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
    )

df_base.head()        # Reviso si la columna 'CustomerDaysInterval' fue creada correctamente.

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,...,Water Area,Time Zone,Product Name,Sales Team,Region,Procurement,Shipping,DeliveryDays,CustomerDays,CustomerDaysInterval
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,...,160302131,America/New York,Dining Furniture,Joshua Bennett,Northeast,151,14,5,19,15-30
1,SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,...,1309517,America/Chicago,Wreaths,Paul Holmes,Midwest,151,22,10,32,30-45
2,SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,...,2254528,America/Detroit,Stemware,Samuel Fowler,Midwest,151,21,10,31,30-45
3,SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,...,3739487,America/New York,Accessories,Carlos Miller,West,151,2,5,7,0-15
4,SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,...,1086045,America/New York,Candles,Joe Price,Northeast,51,16,10,26,15-30


In [8]:
# 5. Utiliza una tabla pivote para mostrar el conteo de órdenes agrupando por Sales Team en las filas y CustomerDaysInterval en las columnas.
# Este reporte debe cuadrar con los datos de la pestaña Reporte1 del archivo reportes.xlsx

pivot_table = pd.pivot_table(                            # Creo la tabla pivote.
              df_base,
              index='Sales Team',
              columns='CustomerDaysInterval',
              values='OrderNumber',
              aggfunc='count',
              fill_value=0,
              observed=False
)

pivot_table.head()       # Reviso los datos para saber si cuadran con los de 'Reporte1'del archivo reportes.xlsx.

CustomerDaysInterval,0-15,15-30,30-45,45-60,60-75,75-90
Sales Team,,,,,,
Adam Hernandez,81,179,42,0,0,0
Anthony Berry,76,181,41,0,0,0
Anthony Torres,81,162,45,0,0,0
Carl Nguyen,96,178,40,0,0,0
Carlos Miller,65,148,34,0,0,0
Chris Armstrong,92,156,44,0,0,0
Donald Reynolds,97,163,36,0,0,0
Douglas Tucker,87,145,45,0,0,0
Frank Brown,85,150,40,0,0,0


In [9]:
# 6. Agrega la siguiente columna calculada a la base:

# KPI                Cálculo
# GrossMargin        OrderQuantity*(UnitPrice*(1-Discount Applied)-UnitCost)

# Visión de Negocio: El margen bruto es un indicador financiero que muestra la diferencia entre
# los ingresos generados por la venta de un producto o servicio y los costos directos asociados
# con su producción o adquisición.

df_base['GrossMargin'] = df_base['Order Quantity'] * (df_base['Unit Price'] * (1 - df_base['Discount Applied']) - df_base['Unit Cost'])

df_base.head()  # Reviso si sumo la columna y no hay errores.

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,...,Time Zone,Product Name,Sales Team,Region,Procurement,Shipping,DeliveryDays,CustomerDays,CustomerDaysInterval,GrossMargin
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,...,America/New York,Dining Furniture,Joshua Bennett,Northeast,151,14,5,19,15-30,4073.4325
1,SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,...,America/Chicago,Wreaths,Paul Holmes,Midwest,151,22,10,32,30-45,886.4100
2,SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,...,America/Detroit,Stemware,Samuel Fowler,Midwest,151,21,10,31,30-45,905.5050
3,SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,...,America/New York,Accessories,Carlos Miller,West,151,2,5,7,0-15,5486.7640
4,SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,...,America/New York,Candles,Joe Price,Northeast,51,16,10,26,15-30,1312.1280


In [10]:
#7. En esta parte debes aplicar un porcentaje a la columna GrossMargin
# pero ese porcentaje dependerá del intervalo en que se encuentre GrossMargin.


# KPI                            Cálculo
# CommissionsPercentage          % de comisión según la tabla de comisiones

# hint: Utiliza pd.cut con labels numéricas para que la columna resultante sea de tipo float.

# Intervalo Comisión
#GrossMargin entre $0 y $100 - 5%
#GrossMargin entre $100 y $1000 -10%
#GrossMargin entre $1000 y $10000 - 15%
#GrossMargin entre $10000 y $100000 - 20%


bins = [0, 100, 1000, 10000, 100000]      # Definimos los límites (bins).
labels = [0.05, 0.10, 0.15, 0.20]         # Comisiones como valores float.


df_base['CommissionsPercentage'] = pd.cut(     # Creamos la columna 'CommissionsPercentage'.
    df_base['GrossMargin'],
    bins=bins,
    labels=labels,
    right=False
).astype(float)    # Aseguramos que sea de tipo 'float'.

df_base.head()    # Reviso si los porcentajes corresponden a lo indicado.


,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,...,Product Name,Sales Team,Region,Procurement,Shipping,DeliveryDays,CustomerDays,CustomerDaysInterval,GrossMargin,CommissionsPercentage
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,...,Dining Furniture,Joshua Bennett,Northeast,151,14,5,19,15-30,4073.4325,0.15
1,SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,...,Wreaths,Paul Holmes,Midwest,151,22,10,32,30-45,886.4100,0.10
2,SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,...,Stemware,Samuel Fowler,Midwest,151,21,10,31,30-45,905.5050,0.10
3,SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,...,Accessories,Carlos Miller,West,151,2,5,7,0-15,5486.7640,0.15
4,SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,...,Candles,Joe Price,Northeast,51,16,10,26,15-30,1312.1280,0.15


In [11]:
#8. Calcula el monto de la comision, utilizando la Tabla del Anexo.

# KPI                       Calculo
# CommissionsAmount         GrossMargin*CommissionsPercentage


df_base['CommissionsAmount'] = df_base['GrossMargin'] * (df_base['CommissionsPercentage'])

df_base.head()    # Reviso si los porcentajes corresponden a lo indicado.


,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,...,Sales Team,Region,Procurement,Shipping,DeliveryDays,CustomerDays,CustomerDaysInterval,GrossMargin,CommissionsPercentage,CommissionsAmount
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,...,Joshua Bennett,Northeast,151,14,5,19,15-30,4073.4325,0.15,611.014875
1,SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,...,Paul Holmes,Midwest,151,22,10,32,30-45,886.4100,0.10,88.641000
2,SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,...,Samuel Fowler,Midwest,151,21,10,31,30-45,905.5050,0.10,90.550500
3,SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,...,Carlos Miller,West,151,2,5,7,0-15,5486.7640,0.15,823.014600
4,SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,...,Joe Price,Northeast,51,16,10,26,15-30,1312.1280,0.15,196.819200


In [12]:
# 9. Calcula la comision sobre el margen bruto.

# KPI               Cálculo
# NetMargin         GrossMargin-CommissionsAmount

df_base['NetMargin'] = df_base['GrossMargin'] - df_base['CommissionsAmount']

df_base.head()    # Reviso si los porcentajes corresponden a lo indicado.

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,...,Region,Procurement,Shipping,DeliveryDays,CustomerDays,CustomerDaysInterval,GrossMargin,CommissionsPercentage,CommissionsAmount,NetMargin
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,...,Northeast,151,14,5,19,15-30,4073.4325,0.15,611.014875,3462.417625
1,SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,...,Midwest,151,22,10,32,30-45,886.4100,0.10,88.641000,797.769000
2,SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,...,Midwest,151,21,10,31,30-45,905.5050,0.10,90.550500,814.954500
3,SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,...,West,151,2,5,7,0-15,5486.7640,0.15,823.014600,4663.749400
4,SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,...,Northeast,51,16,10,26,15-30,1312.1280,0.15,196.819200,1115.308800


In [13]:
#10. Utilizando una tabla pivote, muestra la suma de los valores GrossMargin, NetMargin y
# CommissionAmount agrupando por Sales Team en las filas. Este reporte debe cuadrar con
# los datos de la pestaña Reporte2 del archivo reportes.xlsx


pivot_table_valores = pd.pivot_table(                                         # Creo la tabla pivote.
              df_base,
              index='Sales Team',
              values=['GrossMargin', 'NetMargin', 'CommissionsAmount'],
              aggfunc='sum',
              fill_value=0,
              observed=False
)

pivot_table_valores.head()      # Reviso si los datos corresponden a los de la pestaña Reporte2 del archivo reportes.xlsx.

,CommissionsAmount,GrossMargin,NetMargin
Sales Team,,,
Adam Hernandez,126637.207525,779095.1955,677987.332975
Anthony Berry,124061.181475,767943.9500,654798.341525
Anthony Torres,117962.211675,743901.4355,636758.048825
Carl Nguyen,123843.212050,774781.4675,661874.732450
Carlos Miller,110882.093875,668752.9290,580326.019125
